In [1]:
# uncommet this block in colab

# !pip install 'protobuf>=3.19.5,<3.20.1' 'transformers>=4.26.1' icetk cpm_kernels 'torch>=1.10' gradio

# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# load the model

from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import os

USE_PTUNING = True
BASE_MODEL_PATH="D:\ml\chatglm-6b-int4"
CHECKPOINT_PATH="./output_zero/llm-checkpoints/checkpoint-17900/"
PRE_SEQ_LEN=100

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)

if 'model' in locals():
    del model
torch.cuda.empty_cache()
if USE_PTUNING:
    config = AutoConfig.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, pre_seq_len=PRE_SEQ_LEN)
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, config=config, trust_remote_code=True)
    prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
    new_prefix_state_dict = {}
    for k, v in prefix_state_dict.items():
        if k.startswith("transformer.prefix_encoder."):
            new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
    model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
    model=model.half().cuda()
    model.transformer.prefix_encoder.float()
    model = model.eval()
    print('Load PTUNED model', CHECKPOINT_PATH,'over', BASE_MODEL_PATH)
else:
    model = AutoModel.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    print('Load original model')
    model=model.half().cuda()
    model = model.eval()
torch.cuda.empty_cache()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.c -shared -o C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.so
Kernels compiled : C:\Users\15617\.cache\huggingface\modules\transformers_modules\chatglm-6b-int4\quantization_kernels_parallel.so
Cannot load cpu kernel, don't use quantized model on cpu.
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at D:\ml\chatglm-6b-int4 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load PTUNED model ./output_zero/llm-checkpoints/checkpoint-17900/ over D:\ml\chatglm-6b-int4


In [2]:
chatglm_tokenizer=tokenizer
chatglm_model=model
def llm_chatglm(query):
    count = 0
    old_length=0
    if llm_chatglm.show_query:
        print('\033[91m'+query+'\033[0m',end='')
    if llm_chatglm.disabled:
        return ''
    else:
        if llm_chatglm.max_output_tokens is None:
            max_output_tokens=llm_chatglm.max_all_tokens-len(chatglm_tokenizer.encode(query))
        else:
            max_output_tokens=llm_chatglm.max_output_tokens
        if llm_chatglm.show_response:
            print('\033[94m',end='')
        for response, history in chatglm_model.stream_chat(chatglm_tokenizer, query, history=[],temperature=llm_chatglm.temperature,max_length=llm_chatglm.max_length):
            if llm_chatglm.show_response:
                print(response[old_length:],end='')
                old_length=len(response)
            count += 1
            if count >= max_output_tokens:
                break
        if llm_chatglm.show_response:
            print('\033[0m',end='')
    return response
llm_chatglm.show_query=True
llm_chatglm.show_response=True
llm_chatglm.disabled=False
llm_chatglm.temperature=0.5
llm_chatglm.max_all_tokens=2048
llm_chatglm.max_output_tokens=None
llm_chatglm.max_length=1024
llm=llm_chatglm

In [26]:
def ask(chars,question=None,history='',nSamples=1,by='问',nLines_per_turn=1,played_by_user=None):
  chars=[chars] if isinstance(chars,str) else chars
  for i in range(nSamples):
    new_history=history
    if question:
      new_history=new_history+f'\n{by}：{question}'
    llm_chatglm.show_query=False
    print('\033[91m'+new_history+'\033[0m',end='')
    for iChar,char in enumerate(chars):
      is_user=char==played_by_user
      for j in range(nLines_per_turn if not is_user else 1):
        torch.cuda.empty_cache()
        if is_user and iChar==len(chars)-1:
          break
        print(f'\n\033[91m{char}：\033[0m',end='')
        prompt2=f'\n{char}：'
        if is_user:
          response=input()
          print('\033[91m'+response+'\033[0m',end='')
        else:
          response=llm_chatglm(new_history+prompt2).replace('/n','')
        new_history=new_history+prompt2+response
    print()
  llm_chatglm.show_query=True

def roleplay(char,history,nSamples=1,nLines_per_turn=1,played_by_user=None):
  return ask(char,None,history,nSamples=nSamples,nLines_per_turn=nLines_per_turn,played_by_user=played_by_user)


In [24]:
ask(['希儿菲朵']*4,'请介绍下你自己，和你的身份')


问：请介绍下你自己，和你的身份
希儿菲朵：\me 摇着头
希儿菲朵：我是一只大猫娘们儿。
希儿菲朵：\me 摇晃尾巴
希儿菲朵：(好像很骄傲的样子)


In [27]:
roleplay(['塔芭莎','希儿菲朵']*3,'【美丽的月夜里，塔巴莎骑在希儿菲朵背上看书】\n希儿菲朵：塔芭莎，我好饿好饿好饿\n塔巴莎：\\me 看书',played_by_user=None)

【美丽的月夜里，塔巴莎骑在希儿菲朵背上看书】
希儿菲朵：塔芭莎，我好饿好饿好饿
塔巴莎：\me 看书
塔芭莎：(正在读一本关于生物学的教科书)
希儿菲朵：好厉害啊，塔芭莎
塔芭莎：(自信地回答问题)
希儿菲朵：那，我也可以骑在你的背上看书吗？
塔芭莎：\me 点头
希儿菲朵：\me 骑在希儿菲朵背上


In [29]:
roleplay(['丘鲁克','露易丝']*4,'露易丝：\\me 拔出魔杖，念咒语\n【露易丝的魔法失败引发了大爆炸】',played_by_user=None)

露易丝：\me 拔出魔杖，念咒语
【露易丝的魔法失败引发了大爆炸】
丘鲁克：\me 奔进房间
露易丝：(难道我的魔法失败了吗？)
丘鲁克：\me 扑向露易丝
露易丝：\me 反击
丘鲁克：\me 摔倒
露易丝：(我为什么要这样做呢？)
丘鲁克：\me 昏迷过去
露易丝：(我要报复他！)


In [32]:
roleplay(['露易丝','才人']*5,'''【露易丝发现才人和谢斯塔躲在一个桶里】''' ,played_by_user=None)

【露易丝发现才人和谢斯塔躲在一个桶里】
露易丝：(他们到底躲在哪个桶里呢？)
才人：(我想起来了，谢斯塔可能把桶放在房间的一角，才人可能会躲在旁边。)
露易丝：那他们为什么还穿着长袍呢？
才人：(因为那是为了保护自己不被我发现。)
露易丝：(他们打算干什么呢？)
才人：(我想到了谢斯塔可能会用枪杀我。所以我才没有暴露自己。)
露易丝：那谢斯塔你为什么还穿着长袍呢？
才人：(因为谢斯塔可能用枪杀我。)
露易丝：(他们打算干什么呢？)
才人：(我想起来了，谢斯塔可能把桶放在房间的一角......我可能会在哪个房间里躲起来。)


In [33]:
ask('露易丝','请介绍一下能量守恒原理',nSamples=1)
ask('露易丝','请介绍下魔法的五大系统？',nSamples=2)
ask('露易丝','半夜睡不着怎么办',nSamples=2)


问：请介绍一下能量守恒原理
露易丝：(我才不会告诉你，我也听说过这个呢。)

问：请介绍下魔法的五大系统？
露易丝：(紧张)

问：请介绍下魔法的五大系统？
露易丝：(好难回答啊。)

问：半夜睡不着怎么办
露易丝：(烦恼)

问：半夜睡不着怎么办
露易丝：(困惑地摇了摇头)


In [36]:
roleplay(['丘鲁克','露易丝']*6,'''{丘鲁克拿走了露易丝编织的毛衣，露易丝为了要拿回来而挣扎。谢斯塔向才人表白，但又因为自卑而自我否定。露易丝意外地走进了房间，看到了一些尴尬的场面。丘鲁克向才人展示了一张藏宝地图，计划和才人一起去寻宝并卖掉宝物换钱。大家决定准备出发。}
{丘鲁克 主角之一，才人的朋友，与露易丝关系不好 与才人一起寻宝并卖掉宝物换钱 谢斯塔和基修的争执 向才人展示藏宝地图，决定一起去寻找宝藏}
【丘鲁克 丘鲁克·奥古斯都·菲列特利加·封·安哈尔特·泽鲁普斯特 露易丝的同学，与露易丝关系特别不好，经常取笑露易丝的魔法水平。在后期与露易丝关系有所改善。】
【丘鲁克 她的性格:强气，表面风骚但还是很珍视自己外骚内闷？越生气越冷静 她的兴趣：玩拼图 她的特殊技能: 药草 她讨厌：下雨、露易丝 （小说后来与露易丝很要好） 她的好友：塔芭莎、露易丝】
露易丝：这个可是叫做《始祖的祈祷书》的国宝级的书呢！
丘鲁克：你怎么会有这种国宝级的书呢？
露易丝：我在安莉艾塔的结婚仪式上发表致辞，那个时候要用到《始祖的祈祷书》……等等。
丘鲁克：原来如此，之前去阿比昂也是跟这个公主的婚礼有关的吧？
露易丝：（想起之前丘鲁克为了让自己能够去而甘愿当了诱饵，于是老实地点了点头。）
丘鲁克：也就是说我们时在为公主的结婚典礼平安举行而冒险啰。那不是很名誉的任务吗？也就是说之前特雷斯特因和格马尼亚的同盟也有关系吧？
露易丝：这件事可是对谁也不能说的哦。\emoji 微怒''',played_by_user=None)

{丘鲁克拿走了露易丝编织的毛衣，露易丝为了要拿回来而挣扎。谢斯塔向才人表白，但又因为自卑而自我否定。露易丝意外地走进了房间，看到了一些尴尬的场面。丘鲁克向才人展示了一张藏宝地图，计划和才人一起去寻宝并卖掉宝物换钱。大家决定准备出发。}
{丘鲁克 主角之一，才人的朋友，与露易丝关系不好 与才人一起寻宝并卖掉宝物换钱 谢斯塔和基修的争执 向才人展示藏宝地图，决定一起去寻找宝藏}
【丘鲁克 丘鲁克·奥古斯都·菲列特利加·封·安哈尔特·泽鲁普斯特 露易丝的同学，与露易丝关系特别不好，经常取笑露易丝的魔法水平。在后期与露易丝关系有所改善。】
【丘鲁克 她的性格:强气，表面风骚但还是很珍视自己外骚内闷？越生气越冷静 她的兴趣：玩拼图 她的特殊技能: 药草 她讨厌：下雨、露易丝 （小说后来与露易丝很要好） 她的好友：塔芭莎、露易丝】
露易丝：这个可是叫做《始祖的祈祷书》的国宝级的书呢！
丘鲁克：你怎么会有这种国宝级的书呢？
露易丝：我在安莉艾塔的结婚仪式上发表致辞，那个时候要用到《始祖的祈祷书》……等等。
丘鲁克：原来如此，之前去阿比昂也是跟这个公主的婚礼有关的吧？
露易丝：（想起之前丘鲁克为了让自己能够去而甘愿当了诱饵，于是老实地点了点头。）
丘鲁克：也就是说我们时在为公主的结婚典礼平安举行而冒险啰。那不是很名誉的任务吗？也就是说之前特雷斯特因和格马尼亚的同盟也有关系吧？
露易丝：这件事可是对谁也不能说的哦。\emoji 微怒
丘鲁克：恩，不过我们现在已经无所谓了。现在我们已经找到宝藏了，所以一切都不重要了。
露易丝：什么宝藏啊，都是假的东西。
丘鲁克：是啊，不过这次也应该没问题吧。
露易丝：(安心地点了点头。)
丘鲁克：塔芭莎，我们一起去寻宝吧。
露易丝：好的。
丘鲁克：才人，你也要一起吗？
露易丝：嗯，我需要买的衣服还没有买好呢。
丘鲁克：那就一起吧。
露易丝：(于是两人便一起走出了房间。)
丘鲁克：塔芭莎，我们一起去寻宝吧。
露易丝：好的。
